In [1]:
from bs4 import BeautifulSoup
import urllib.request
import re

import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import cv2 as cv2
import matplotlib.pyplot as plt
%matplotlib inline

from urllib.request import Request, urlopen
from random import randint,shuffle
import time
from time import sleep


Using TensorFlow backend.


In [2]:
!pip install user_agent
import user_agent
from user_agent import generate_user_agent, generate_navigator

In [0]:
content = []
skysports = [
       "https://www.skysports.com/football/transfer-news",
       "https://www.skysports.com/football/transfer-news/more/2",
       "https://www.skysports.com/football/transfer-news/more/3",
       "https://www.skysports.com/football/transfer-news/more/4",
       "https://www.skysports.com/football/transfer-news/more/5",
       "https://www.skysports.com/football/transfer-news/more/6",
       "https://www.skysports.com/football/transfer-news/more/7",
       "https://www.skysports.com/football/transfer-news/more/8",
       "https://www.skysports.com/football/transfer-news/more/9",
       "https://www.skysports.com/football/transfer-news/more/10",
       "https://www.skysports.com/football/transfer-news/more/11",
       "https://www.skysports.com/football/transfer-news/more/12"
       ]
for url in skysports:
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')
    regex = re.compile('news-item')
    content_lis = soup.find_all('div', attrs={'class': regex})
    for div in content_lis:
        title = div.find('h2').getText()
        link = div.find('a')["href"]
        x = re.search("^/transfer" , link)
        if x != None :
          link = "https://www.skysports.com" + link;
        topic = div.find('strong').getText().strip()
        newspage = urllib.request.urlopen(link)
        tsoup = BeautifulSoup(newspage, 'html.parser')
        try:
          article_title = tsoup.find('h1').getText()
        except:
          article_title = ""
        try:
          article_subtitle = tsoup.find('h2').getText()
        except:
          article_subtitle = ""
        h3_array = tsoup.find_all('h3')
        article_part = []
        for h3 in h3_array : 
            article_part.append(h3.getText())
        article_part = '\n'.join(article_part)
        p_array = tsoup.find_all('p')
        article_content = []
        for h3 in p_array : 
            article_content.append(h3.getText())
        article_content = '\n'.join(article_content)
        regex = re.compile("highlight")
        try:
          date = tsoup.find("span" , attrs={"class":regex}).getText()
        except:
          date = ""
        article = {
            "url" : link ,
            "subscript" : title ,
            "topic" : topic ,
            "title" : article_title ,
            "subtitle" : article_subtitle ,
            "emphasis" : article_part ,
            "content" : article_content ,
            "date" : date
        }
        content.append(article)
df = pd.DataFrame(content)
df.to_csv("skysport_data.csv")

In [0]:
headers = {'User-Agent': generate_user_agent()}
content = []
links = []
goal = [
        "https://www.goal.com/en-us/transfer-news"
       ]
for i in range(2,501):
  new_url = goal[0]+"/"+str(i)
  goal.append(new_url)
shuffle(goal)

In [0]:
import time
n = 0
program_starts = time.time()
for url in goal:
  print(url)
  now = time.time()
  print("{0} sites have been scraped .It has been {1} seconds".format(n,now - program_starts))
  sleep(randint(1,30))
  request = Request(url, headers=headers)
  page = urllib.request.urlopen(request)
  soup = BeautifulSoup(page, 'html.parser')
  content_lis = soup.find_all('article')
  for article in content_lis :
    regex = re.compile("article")
    link = article.find('a', attrs={'class': regex})
    if link != None:
      link = "https://www.goal.com"+link["href"]  
      links.append(link)
  n = n + 1

In [0]:
links = pd.DataFrame(links)
links.to_csv("goal.csv")
links

In [0]:
df = pd.read_csv("skysport_data.csv")
links = pd.read_csv("goal.csv")
links = list(links["0"])

In [0]:
shuffle(links)
links

In [0]:
headers = {'User-Agent': generate_user_agent()}
n = 0
program_starts = time.time()
for url in links:
  try:
    n = n+1
    request = Request(url, headers=headers)
    page = urllib.request.urlopen(request)
    soup = BeautifulSoup(page, 'html.parser')
    sleep(randint(1,10))
    try:
      article_title = soup.find("h1").getText()
    except:
      article_title = ""
    try:
      regex = re.compile("teaser")
      article_subtitle = soup.find("div",attrs={"class" : regex}).getText()
    except:
      article_subtitle = ""
    h3_array = soup.find_all('h2')
    article_part = []
    for h3 in h3_array : 
        article_part.append(h3.getText())
    article_part = '\n'.join(article_part)
    p_array = soup.find_all('p')
    article_content = []
    for h3 in p_array : 
        article_content.append(h3.getText())
    article_content = '\n'.join(article_content)
    regex = re.compile("tags-list__link")
    taglist = soup.find_all("a",attrs={"class" : regex})
    tags = []
    for tag in taglist:
      tags.append(tag.getText())
    topic = '+'.join(tags)
    datetag = soup.find("time")
    date = datetag.getText()
    article = {
              "url" : url ,
              "subscript" : article_title ,
              "topic" : topic ,
              "title" : article_title ,
              "subtitle" : article_subtitle ,
              "emphasis" : article_part ,
              "content" : article_content ,
              "date" : date.strip()
          }
    df = df.append(article , ignore_index=True)  
    df.to_csv("transfer_data.csv")
    now = time.time()
    print("{0} sites have been scraped .It has been {1} seconds".format(n,now - program_starts))
  except:
    n=n-1

1 sites have been scraped .It has been 5.2439353466033936 seconds
2 sites have been scraped .It has been 16.342480421066284 seconds
3 sites have been scraped .It has been 19.604100704193115 seconds
4 sites have been scraped .It has been 28.873061180114746 seconds
5 sites have been scraped .It has been 37.14517688751221 seconds
6 sites have been scraped .It has been 47.252798318862915 seconds
7 sites have been scraped .It has been 53.40668869018555 seconds
8 sites have been scraped .It has been 64.64592170715332 seconds
9 sites have been scraped .It has been 71.81889486312866 seconds
10 sites have been scraped .It has been 82.93910884857178 seconds
11 sites have been scraped .It has been 85.11512780189514 seconds
12 sites have been scraped .It has been 90.290598154068 seconds
13 sites have been scraped .It has been 98.49788856506348 seconds
14 sites have been scraped .It has been 100.89242267608643 seconds
15 sites have been scraped .It has been 104.97694969177246 seconds
16 sites have 

In [0]:
df.to_csv("final.csv")

In [0]:
df

,url,subscript,topic,title,subtitle,emphasis,content,date
0,https://www.skysports.com/transfer/news/11095/...,FIFA reviews coronavirus impact on contracts,Transfer headlines,\n Coronavirus: FIFA reviewing impa...,Transfer Centre,Home\nCoronavirus - key sporting developments\...,\n\n ...,16/03/20 4:02pm
1,https://www.skysports.com/transfer/news/11667/...,Man Utd trigger option on Matic contract,Latest Done Deals,\n Nemanja Matic: Manchester United...,Transfer Centre,News\nAnalysis: The resurgence of Matic\nJoin ...,\nLast Updated: 17/03/20 4:14pm\n\nNemanja Mat...,17/03/20 4:14pm
2,https://www.skysports.com/transfer/news/12691/...,Where next for Gareth Bale?,Real Madrid,\n Gareth Bale: Where next for Real...,Transfer Centre,News\nPremier League\nSerie A\nMajor League So...,\nLast Updated: 21/03/20 11:36am\n\nGareth Bal...,21/03/20 11:36am
3,https://www.skysports.com/transfer/news/12691/...,Arsenal transfer rumours,Arsenal,\n Arsenal transfer news and rumour...,Transfer Centre,News\nThe latest players linked with a move to...,\nBy Sky Sports Football |\nLast Updated: 21/0...,21/03/20 10:57am
4,https://www.skysports.com/transfer/news/12691/...,Man City transfer rumours,Manchester City,\n Man City transfer news and rumou...,Transfer Centre,News\nThe latest players linked with a move to...,\nBy Sky Sports Football |\nLast Updated: 21/0...,21/03/20 10:06am
...,...,...,...,...,...,...,...,...
135,https://www.goal.com/en-us/news/atlanta-united...,Atlanta United signs Tottenham prospect Walkes...,Transfers + MLS + Premier League + Atlanta Un...,Atlanta United signs Tottenham prospect Walkes...,The 19-year-old center back is joining the ML...,Trending News,Other English editions\nAtlanta United has sig...,1/26/17
136,https://www.goal.com/en-us/news/marotta-to-sta...,Marotta to stand down as Juventus CEO,Juventus + Juventus v Napoli + Transfers + Se...,Marotta to stand down as Juventus CEO,After eight years at the Serie A champions is...,Trending News\nRelated,Other English editions\nGiuseppe Marotta has c...,18:03 9/29/18
137,https://www.goal.com/en-us/news/he-will-never-...,'He will never leave' - Bartomeu claims 'geniu...,Barcelona + Lionel Messi + Primera División +...,'He will never leave' - Bartomeu claims 'geniu...,The Spanish giants may already be preparing f...,Trending News,Other English editions\nBarcelona president Jo...,5/2/19
138,https://www.goal.com/en-us/news/man-utd-and-aj...,Man Utd and Ajax agree £14m Blind fee as defen...,Transfers + Ajax + Premier League + Mancheste...,Man Utd and Ajax agree £14m Blind fee as defen...,The Eredivisie club have announced they have ...,Trending News,Other English editions\nManchester United and ...,7/17/18
